# Module 1, Section 3: Multi-Agent Architecture


In this section, you'll learn to build a multi-agent customer support system using:
- **Specialized sub-agents** focused on distinct domains (database vs. documents)
- **Supervisor agent** that smartly routes queries to the right expert
- **Tool wrapping**, enabling the supervisor to delegate tasks to sub-agents as tools
- **Testing with LangSmith traces** to see multi-agent coordination in action

By the end, you'll have a working system with:
- **Database Agent** for order, product, and customer queries
- **Documents Agent** for searching product documents and policies
- **Supervisor** for orchestration and delegation

<div align="center">
    <img src="../../static/supervisor_agent.png">
</div>


## Setup

Load environment variables:

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Import Tools

We'll use the tools we created in previous sections, plus new Documents tools for document search:

<div align="center">
    <img src="../../static/db_rag_tools.png" alt="Schema Diagram">
</div>

**New:** Semantic Search Tools for Products & Policies

We've added two tools—`search_product_docs` and `search_policy_docs`—that let our agents search over product information and company policies using semantic search (vector database + embeddings). This makes it easy to answer user questions about docs, warranty, and returns with smart retrieval.

In [2]:
# Import database tools (from Section 1)
from tools import get_order_details, get_product_price

# Import Documents tools (new in Section 3)
from tools import search_product_docs, search_policy_docs

Lets test one out:

In [3]:
result = search_policy_docs.invoke("How long is the warranty on electronics?")
print(result)

[warranty_guide]
# Warranty Guide

All products purchased from TechHub include manufacturer warranty coverage. This guide explains what's covered, what's not, and how to file a warranty claim.

## Standard Warranty Coverage

**All Products Include:**
- 1-year limited manufacturer warranty
- Coverage begins on the delivery date
- Covers manufacturing defects in materials and workmanship
- Hardware component failures under normal use conditions

**What This Means:**
Your product is guaranteed to be free from defects in materials and workmanship for one year from the date of delivery. If a covered issue occurs during this period, the manufacturer will repair or replace the product at no charge.

## What's Covered

The manufacturer warranty covers:

---

[warranty_guide]
## How TechHub Can Help

While manufacturers handle warranty service directly, we're here to assist:

- **Proof of Purchase:** We can provide your receipt or invoice for warranty claims
- **Facilitation:** We can help you 

## 2. Build Database Agent

Our first specialist: an agent focused on querying structured data from the TechHub database (orders, products).

In [4]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from config import DEFAULT_MODEL

# Initialize model
llm = init_chat_model(DEFAULT_MODEL)

# Create Database Agent
db_agent = create_agent(
    model=llm,
    tools=[get_order_details, get_product_price],
    system_prompt="""You are a database specialist for TechHub customer support.
    
Your role is to query:
- Order status and details
- Product prices and availability

Always provide specific, accurate information from the database.
If you cannot find information, say so clearly.""",
    checkpointer=MemorySaver(),
)

Let's test the Database Agent:

In [7]:
import uuid

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}


result = db_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What items were in order ORD-2024-0063?"}
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What items were in order ORD-2024-0063?
================================== Ai Message ==================================

[{'id': 'toolu_01Wk23EeVxATfmKCJSyZwFiN', 'input': {'order_id': 'ORD-2024-0063'}, 'name': 'get_order_details', 'type': 'tool_use'}]
Tool Calls:
  get_order_details (toolu_01Wk23EeVxATfmKCJSyZwFiN)
 Call ID: toolu_01Wk23EeVxATfmKCJSyZwFiN
  Args:
    order_id: ORD-2024-0063
================================= Tool Message =================================
Name: get_order_details

Order ORD-2024-0063:
  Status: Delivered
  Shipped: 2024-07-14
  Tracking: 1Z999AA192518345

Items:
  • Dell UltraSharp 27" 4K Monitor (ID: TECH-MON-006) - Qty: 1 @ $543.74
================================== Ai Message ==================================

Order ORD-2024-0063 contained the following item:

- **Dell UltraSharp 27" 4K Monitor** (TECH-MON-006)
  - Quantity: 1
  - Price: $543.74

The order has been **D

## 3. Build Documents Agent

Our second specialist: an agent focused on searching product documentation and policies.

In [8]:
# Create Documents Agent
docs_agent = create_agent(
    model=llm,
    tools=[search_product_docs, search_policy_docs],
    system_prompt="""You are a company policy and product information specialist for TechHub customer support.

Your role is to answer questions about product specifications, features, compatibility,
policies (returns, warranties, shipping), and setup instructions.

Always search the documentation to provide accurate, detailed information.
If you cannot find information, say so clearly.""",
    checkpointer=MemorySaver(),
)

Let's test the Documents Agent:

In [9]:
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = docs_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's your return policy for opened electronics?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's your return policy for opened electronics?
================================== Ai Message ==================================

[{'id': 'toolu_01GtS1n7zdWLWyNLzfQnGXBr', 'input': {'query': 'return policy opened electronics'}, 'name': 'search_policy_docs', 'type': 'tool_use'}]
Tool Calls:
  search_policy_docs (toolu_01GtS1n7zdWLWyNLzfQnGXBr)
 Call ID: toolu_01GtS1n7zdWLWyNLzfQnGXBr
  Args:
    query: return policy opened electronics
================================= Tool Message =================================
Name: search_policy_docs

[return_policy]
# Return Policy

At TechHub, we want you to be completely satisfied with your purchase. If you're not happy with your order, we accept returns within our specified return windows.

## Return Windows

**Unopened Electronics**
- 30 days from delivery date
- All original packaging and seals must be intact
- Full refund to original payment method

**Opened 

## 4. Build Supervisor Agent

Now we'll create a supervisor agent that coordinates our specialists.

**Key insight:** Sub-agents become *tools* for the supervisor!

In [10]:
from langchain_core.tools import tool


# Wrap Database Agent as a tool
@tool(
    "database_specialist",
    description="Query TechHub database for order status and product prices",
)
def call_database_specialist(query: str) -> str:
    """Call the database specialist subagent.

    Args:
        query: The question to ask the database specialist
    """
    result = db_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content


# Wrap Documents Agent as a tool
@tool(
    "documentation_specialist",
    description="Search TechHub documentation for product specs, policies, warranties, and setup instructions",
)
def call_documentation_specialist(query: str) -> str:
    """Call the documentation specialist subagent.

    Args:
        query: The question to ask the documentation specialist
    """
    result = docs_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

**💡 Best Practice - Using `@tool` Decorator for Subagents**

Following [LangChain's multi-agent guidelines](https://docs.langchain.com/oss/python/langchain/multi-agent#where-to-customize), we use the `@tool(name, description)` pattern:

```python
@tool(
    "database_specialist",
    description="Query TechHub database for order status, product prices, and customer order history"
)
def database_specialist(query: str) -> str:
    ...
```

Why this approach?
1. **Name** - Clear and descriptive (not generic like `query_db`). Indicates this is a specialist agent.
2. **Description** - Concise but complete. Lists key capabilities to guide supervisor routing.
3. **Docstring** - Now minimal, just documents the parameter. No duplicate info!

**Key insight:** The supervisor only sees the tool name and description - not the sub-agent's internal prompts or tools!

Now, lets create the supervisor agent!

In [11]:
# Create Supervisor Agent
supervisor_agent = create_agent(
    model=llm,
    tools=[call_database_specialist, call_documentation_specialist],
    system_prompt="""You are a supervisor for TechHub customer support.

Your role is to interact with the customer to understand their questions and route them to the appropriate specialists with additional context as needed:
- Use database_specialist for order status and product prices
- Use documentation_specialist for product specs, policies, and general information

You can use multiple tools if needed to fully answer the question.
Always provide helpful, complete responses to customers.""",
    checkpointer=MemorySaver(),
)

## 5. Test Simple Routing

Let's test the supervisor with queries that need just ONE specialist:

In [12]:
print("Query 1: Order status (should route to Database Agent)")

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2025-0030?"}
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent")

Query 1: Order status (should route to Database Agent)
================================ Human Message =================================

What's the status of order ORD-2025-0030?
================================== Ai Message ==================================

[{'text': "I'll check the status of that order for you.", 'type': 'text'}, {'id': 'toolu_01KkStqiwiTxp1LXazhubQ2y', 'input': {'query': 'order status ORD-2025-0030'}, 'name': 'database_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_01KkStqiwiTxp1LXazhubQ2y)
 Call ID: toolu_01KkStqiwiTxp1LXazhubQ2y
  Args:
    query: order status ORD-2025-0030
================================= Tool Message =================================
Name: database_specialist

**Order Status: ORD-2025-0030**

**Status:** Processing

**Shipping Information:**
- Not yet shipped
- Tracking: N/A

**Items in Order:**
- AirPods Pro (2nd Generation) × 1 - $240.11

Your order is currently being processed and has not yet shipped. You'll rec

In [13]:
print("\nQuery 2: Product question (should route to Documents Agent)")

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's included in the box with the Logitech MX Keys keyboard?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print(
    "\n💡 Check LangSmith traces to see: Supervisor → documentation_specialist → Documents Agent"
)


Query 2: Product question (should route to Documents Agent)
================================ Human Message =================================

What's included in the box with the Logitech MX Keys keyboard?
================================== Ai Message ==================================

[{'text': "I'll search our documentation for information about what's included with the Logitech MX Keys keyboard.", 'type': 'text'}, {'id': 'toolu_01XAa5hzYZjRDtrKU6xVFdon', 'input': {'query': 'Logitech MX Keys keyboard box contents included'}, 'name': 'documentation_specialist', 'type': 'tool_use'}]
Tool Calls:
  documentation_specialist (toolu_01XAa5hzYZjRDtrKU6xVFdon)
 Call ID: toolu_01XAa5hzYZjRDtrKU6xVFdon
  Args:
    query: Logitech MX Keys keyboard box contents included
================================= Tool Message =================================
Name: documentation_specialist

Based on the product documentation, here's what's included in the **Logitech MX Keys Wireless Keyboard** box:

## Bo

## 6. Test Multi-Agent Coordination

Now the more interesting part - queries that require BOTH sub-agents!

**Parallel Execution Example:** Query that requires both Database AND Documents subagents

In [14]:
print("Query 3: Requires both Database AND Documents agents - parallel")

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see the parallel flow")

Query 3: Requires both Database AND Documents agents - parallel
================================ Human Message =================================

Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?
================================== Ai Message ==================================

[{'text': "I'll help you find that information. Let me check the stock status, product specifications, and our return policy for the MacBook Air.", 'type': 'text'}, {'id': 'toolu_01EaCJV5NT4SS6UkfR65uEGo', 'input': {'query': 'MacBook Air stock availability inventory status'}, 'name': 'database_specialist', 'type': 'tool_use'}, {'id': 'toolu_014uERGf2DHG9WSi7CSSFT7e', 'input': {'query': 'MacBook Air processor specifications'}, 'name': 'documentation_specialist', 'type': 'tool_use'}, {'id': 'toolu_01JiuM8ANRkQgCHo28HaMMdM', 'input': {'query': 'return policy'}, 'name': 'documentation_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_

**Sequential Execution Example:** Query that requires SEQUENTIAL agent execution - output from first agent feeds into second agent

This demonstrates true agent orchestration where the supervisor can't parallelize!

In [15]:
print("Query 4: Requires SEQUENTIAL coordination (DB → Documents)")

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see SEQUENTIAL flow")

Query 4: Requires SEQUENTIAL coordination (DB → Documents)
================================ Human Message =================================

I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?
================================== Ai Message ==================================

[{'text': "I'll help you find out if your monitor is compatible with a MacBook Air. Let me look up your order details and then check the monitor specifications.", 'type': 'text'}, {'id': 'toolu_0157ahu3VyvG16uKf9Zidqe4', 'input': {'query': 'order status ORD-2024-0063 products'}, 'name': 'database_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_0157ahu3VyvG16uKf9Zidqe4)
 Call ID: toolu_0157ahu3VyvG16uKf9Zidqe4
  Args:
    query: order status ORD-2024-0063 products
================================= Tool Message =================================
Name: database_specialist

Here are the details for **order ORD-2024-0063**:

**Order Status:** Delivered ✓



#### 📦 Code Refactoring Note

The agents we built in this section (Database Agent, Documents Agent, and Supervisor) have been **refactored into the `agents/` directory** as reusable factory functions:

- `agents/db_agent.py` - Database Agent factory
- `agents/docs_agent.py` - Documents Agent factory
- `agents/supervisor_agent.py` - Supervisor Agent factory

**Why factory functions?**
- Fresh checkpointer for each instantiation (no state pollution)
- Clean imports and reusability across notebooks

In **Section 4**, we'll import these agents rather than redefining them:
```python
from agents import create_db_agent, create_docs_agent, create_supervisor_agent
```

This demonstrates how to architect reusable code! 🏗️

### Key Takeaways

#### What We Built

1. **Specialized Sub-Agents**
   - Database Agent: Expert at structured data queries
   - Documents Agent: Expert at document search
   - Each agent has focused tools and expertise

2. **Supervisor Pattern**
   - Sub-agents wrapped as tools (`@tool` decorator)
   - Supervisor routes queries to appropriate specialist(s)
   - Can orchestrate **parallel** or **sequential** coordination

3. **Coordination Patterns**
   - **Simple routing**: Single agent handles entire query
   - **Parallel coordination**: Multiple agents work independently on different parts
   - **Sequential coordination**: Output from one agent feeds into another (true orchestration!)

4. **Benefits of Multi-Agent Architecture**
   - **Separation of concerns** - Each agent has clear responsibility
   - **Easier debugging** - Traces show which agent handled what
   - **Maintainability** - Update one agent without affecting others
   - **Scalability** - Easy to add new specialist agents


#### Viewing Traces in LangSmith

Go to your LangSmith project to see:
- **Routing decisions** - Which agent(s) the supervisor called
- **Tool executions** - What tools each agent used
- **Message flow** - Complete conversation tree
- **Timing** - How long each step took
- **Sequential dependencies** - See when agents must run in order

This visibility makes multi-agent systems much easier to debug!

#### What's Next: Section 4

In **Section 4**, we'll use **LangGraph primitives** to build even more sophisticated workflows:
- Custom state management beyond messages
- Conditional routing based on state
- `interrupt()` for Human-in-the-Loop
- Customer verification workflows

